# Résumer un document

Commencez par télécharger le document que vous souhaitez résumer.

In [1]:
# Download a file
import os

import wget

file_path = "my_document.pdf"
if os.path.exists(file_path):
    os.remove(file_path)
doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

wget.download(doc_url, out=file_path)

'my_document.pdf'

Puis instancier la connexion à l'API Albert.

In [2]:
# OpenAI client configuration
import requests
from openai import OpenAI

base_url = "https://albert.api.etalab.gouv.fr/v1"
api_key = os.getenv("API_KEY")

client = OpenAI(base_url=base_url, api_key=api_key)

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}

Enfin pour vous importer le document dans une collection de notre base vectorielle à l'aide du endpoint POST `/v1/files`.

Vous devez spécifier le modèle d'embeddings qui sera utilisé pour vectoriser votre document. Vous pouvez trouver la liste des modèles avec le endpoint `/v1/models`. Les modèles d'embeddings sont indiqués avec le type _feature-extraction_.

Le endpoint POST `/v1/files` doit retourner un status _success_. 

In [3]:
# Remove previous files

collection = "leo"
response = session.delete(f"{base_url}/files/{collection}")
response.status_code

204

In [4]:
# Upload a file
collection = "leo"
model = "BAAI/bge-m3"
params = {"collection": collection, "model": model} 

files = {'files': (os.path.basename(file_path), open(file_path, 'rb'), "application/pdf")}
response = session.post(f"{base_url}/files", params=params , files=files)

response.json()

{'object': 'list',
 'data': [{'object': 'upload',
   'id': '0d18a01a-d921-462c-a1c6-ce7c650ebf15',
   'filename': 'my_document.pdf',
   'status': 'success'}]}

Vous pouvez observer les fichiers que vous avez importer dans une collection à l'aide du endpoint GET `/v1/files`.

In [5]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")

response.json()

{'object': 'list',
 'data': [{'object': 'file',
   'id': '0d18a01a-d921-462c-a1c6-ce7c650ebf15',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'chunk_ids': ['0162a0fc-ddc2-4dd0-8835-89f9fb218e0a',
    '0b9305f3-dc49-4011-b789-a03c6b13e49a',
    '1c0b8d3b-f0c3-4c51-95b4-baee4cf40c18',
    '440fcb9f-73de-4135-b47b-dcd1bdf7f9e4',
    '56fd9c7d-29bd-4c6b-9d4c-40fe123b7e20',
    '586852d9-3573-4756-b242-ab52c4769288',
    '5a75b874-46b8-4c79-941b-8f1b40dcc88e',
    '8df31ab5-fba8-485f-9721-9ad487e6b265',
    '9cbe7e5f-aefd-4dfe-b6bc-58e774ff7d1a',
    '9e59e8dd-6e31-41a7-9980-94436307e1ee',
    'a1d3daad-feec-4ffd-a611-ab44db18696d',
    'cd9b4a1a-974c-45ae-a908-8132e735937e',
    'e335df17-510c-4757-97b1-21be3fd1f209',
    'e777305d-a5bf-4488-94bf-01b9c1e8126c',
    'f30fd131-cd56-4b77-ad97-f01cfb7ebd39',
    'fc1fc8cf-02b4-4178-ae1c-5486bff75212'],
   'created_at': 1722840171}]}

Chaque fichier importé est associé un _file_id_, stocker le, il sera nécessaire pour la suite.

In [6]:
file_id = response.json()["data"][0]["id"]

Pour résumer un document vous devez utiliser le _tool_ qui permet de fournir au modèle un ou plusieurs document préalablement importés : _UseFile_. Vous trouverez la liste des _tools_ ainsi que leur paramètres avec le endpoint GET `/v1/tools`.

In [7]:
# Display tools parameters
response = session.get(f"{base_url}/tools")
for tool in response.json()["data"]:
    if tool["id"] == "UseFiles":
        print(tool["description"])

Fill your prompt with file contents. Your prompt must contain "{files}" placeholder.

    Args:
        collection (str): Collection name.
        file_ids (List[str]): List of file ids in the selected collection.


Nous pouvons maintenant interroger le modèle avec le endpoint POST `/v1/chat/completions`

In [8]:
# Chat completions
message = "Résume le document suivant : {files}"
user = "leo" # for chat history
data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "user": user,
    "messages": [{"role": "user", "content": message}],
    "stream": False,
    "n": 1,
    "tools": [
        {
            "function": {
                "name": "UseFiles",
                "parameters": {
                    "collection": collection,
                    "file_ids": [file_id]
                },
            },
            "type": "function",
        }
    ],
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)

Le document est une décision du 18 juin 2023 portant délégation de signature pour la direction interministérielle du numérique. Voici les principaux points :

* La décision est publiée au Journal officiel de la République française.
* La direction interministérielle du numérique est dirigée par Mme Stéphanie Schaer.
* La décision délègue la signature de certains actes à des agents suivants :
	+ M. Perica Sucevic et Mme Floriane Beaudron pour les demandes d'achat et les attestations électroniques.
	+ Mme Elsa Le Duigou, M. Valentin Brel, Mme Margot Sanchez, M. Paul Burgun et Mme Emma Ghariani pour signer les actes relevant des attributions de leurs pôles.
	+ Mme Géraldine Taurand pour signer les actes de gestion et procéder aux validations électroniques.
	+ Les agents suivants pour signer les services faits relevant des attributions de leurs départements :
		- M. Gilles Corbi, M. Yann Brûlé, M. Ulrich Tan, Mme Marie-Christie Ritz, Mme Fadila Leturcq, Mme Marielle Variette, M. Aël Lebret

Vous pouvez récupérer les metadata des tools utilisées. Ces metadata vous donnera, entre autre, le prompt envoyé au modèle.

In [11]:
print(response.metadata[0]["UseFiles"]["prompt"])

Résume le document suivant : Art.  8.  –  La présente décision sera publiée au Journal officiel de la République française. Fait le 18 juin 2023. S. SCHAER

informatique  CHORUS  formulaires  des  dépenses  et  recettes  de  la  direction  interministérielle  du  numérique résultant d’actes validés selon les modalités fixées aux articles 1er  à 8 de la présente décision.

Art.  6.  –  Délégation est donnée à l’effet de valider, de façon électronique dans l’outil informatique « Chorus Formulaires »,  dans  les  limites  de  leurs  attributions,  les  demandes  d’achat  et  les  attestations  du  service  fait  aux agents ci-après désignés :

–  M. Perica Sucevic, agent contractuel, préfigurateur de la mission « Droit et international » ; –  Mme Floriane Beaudron, agente contractuelle, cheffe de la mission « Communication ». Art.  3.  –  Délégation est donnée à l’effet de signer au nom du ministre de la transformation et de la fonction publiques ainsi qu’au nom de la Première ministre, t